# **6.047 | 6.878 | HST.507**

#**Fall 2021 Problem Set 5: Clustering Phylogenetic Trees**

Due: Friday, November 19 at 11:59PM (submit copy of notebook to Stellar)

#Load libraries


Feel free to import any additional libraries here.

In [2]:
from google.colab import files

# Clustering Phylogenetic Trees

We've seen that phylogenetic tree algorithms can construct *many* different "good" putative evolutionary histories. An important and challenging problem is that of reducing a large number of trees to a smaller number of representative solutions. The objective of this assignment is to explore techniques for dealing
with large sets of different phylogenetic trees for the same data. This assignment is inspired by the paper [Statistically based postprocessing of phylogenetic analysis by clustering](https://www.ncbi.nlm.nih.gov/pubmed/12169558.) by Cara Stockham, Li-San Wang, and Tandy Warnow (Bioinformatics, Vol 18, Suppl. 1, 2002, pp. S285-S293). This problem is due
to Ran Libeskind-Hadas. The full details are available at the URL below:

http://www.cs.hmc.edu/~hadas/mitcompbio/treedistance.html

You will need to download the following [files](https://www.dropbox.com/sh/94bcsf8hxer11fq/AABqiZjDqHgMHbwJhor7hR5Ja?dl=0) and upload them to colab using the following code block.

In [3]:
caesal = files.upload()
from Caesal import *

Saving Caesal.py to Caesal.py


A) Implement the *Robinson-Foulds* distance metric.

In [72]:
import random
"""
Returns distance between two phylogenetic trees

Arguments:
    tree1 - first list of lists or tuple of tuples, with integer member elements
    tree2 - second list of lists or tuple of tuples, with integer member elements
Returns:
    dist - integer distance between 'tree1' and 'tree2'
    """
class Node(object):
    def __init__(self,value,parent=None) -> None:
        self.value=value
        self.parent = parent
        self.left = None
        self.right = None

    def traverse(self):
        left = self.left.traverse() if self.left else []
        right = self.right.traverse() if self.right else []
        if self.left is None and self.right is None:
          return left + [self.value] + right
        else:
          return left+right

def construct(tree,parent=None):
    x=Node(value=None,parent=parent)
    if type(tree[0]) is int:
      x.left=Node(value=tree[0],parent=x)
    else:
      x.left=construct(tree[0],parent=x)

    if type(tree[1]) is int:
      x.right=Node(value=tree[1],parent=x)
    else:
      x.right=construct(tree[1],parent=x)
    return x

def partitions(tree, bits, d=None):
    if d is None:
      d={}
    label=None
    if tree.left.value is not None and tree.right.value is not None:
      label=bits[tree.left.value]^bits[tree.right.value]
    elif tree.left.value is not None:
      subtree_label, d=partitions(tree.right,bits,d)
      label=bits[tree.left.value]^subtree_label
    elif tree.right.value is not None:
      subtree_label, d=partitions(tree.left,bits,d)
      label=subtree_label^bits[tree.right.value]
    else:
      left_subtree_label, d=partitions(tree.left,bits,d)
      right_subtree_label, d=partitions(tree.right,bits,d)
      label=left_subtree_label^right_subtree_label
    d.setdefault(label,0)
    d[label]+=1
    return label, d


def distance(tree1, tree2):
    dist=0
    t1=construct(tree1)
    t2=construct(tree2)

    #label each leaf
    agenda=t1.traverse()
    lookup={}
    for i in agenda:
      lookup[i]=random.randint(1,100000)

    #find partitions each tree and compute differences
    _,d=partitions(t1,lookup)
    _,d2=partitions(t2,lookup)
    for key in d2:
      if key not in d:
        dist+=1

    return dist


These are the test cases for your `distance` function.

In [18]:
def test_distance():
    T1 = (0, (1, (2, (3, 4))))
    T2 = (0, (1, (3, (2, 4))))
    print("Expected: 1\tOutput: " + str(distance(T1, T2)))
    T3 = (0, ((1, (2, 3)), (7, (6, (4, 5)))))
    T4 = (0, ((2, (1, 3)), (6, (4, (5, 7)))))
    print("Expected: 3\tOutput: " + str(distance(T3, T4)))
    print("Expected: 8\tOutput: " + str(distance(treeList[0], treeList[42])))
    print("Expected: 3\tOutput: " + str(distance(treeList[57], treeList[77])))
    print("Expected: 1\tOutput: " + str(distance(treeList[192], treeList[200])))
    print("Expected: 4\tOutput: " + str(distance(treeList[386], treeList[234])))

In [47]:
test_distance()

Expected: 1	Output: 1
Expected: 3	Output: 3
Expected: 8	Output: 8
Expected: 3	Output: 3
Expected: 1	Output: 1
Expected: 4	Output: 4


B) You will use a clustering algorithm to partition distinct phylogenetic trees into clusters of trees where the trees in a given cluster are similar with respect to *Robinson-Foulds* distance. (Hint: take note of their hint for speeding up the *Robinson-Foulds* distance calculations!)

In [44]:
"""
Return each clustering step from 'maxClusters' to 'minClusters' size from input 'treeList'

Arguments:
    treeList - list of all trees to cluster
    minClusters - integer size of minimum number of clusters
    maxClusters - integer size of maximum number of clusters
Returns:
    clusterings - list of clusters with information on numbers of trees, diameters (Robinson-Foulds distance), and average diameter
        [[[numbers of trees], [diameters], average diameter],   <- max number of clusters
         .
         .
         .
         [[numbers of trees], [diameters], average diameter]]   <- min number of clusters
"""
def cluster(treeList, minClusters, maxClusters):
    #make agenda (clusters), pairwise distances between clusters, and store all pairwise tree distances
    agenda=[[i,[i],0] for i in treeList]
    dist={}
    d=[]
    for i in range(len(treeList)):
      for j in range(len(treeList)):
        dist.setdefault(treeList[i],{})
        dist[treeList[i]][treeList[j]]=distance(treeList[i],treeList[j])
        dist.setdefault(treeList[j],{})
        dist[treeList[j]][treeList[i]]=distance(treeList[i],treeList[j])
        if i!=j:
          d.append([(treeList[i],treeList[j]),[treeList[i],treeList[j]],distance(treeList[i],treeList[j])])

    d.sort(reverse=True,key=lambda x: x[2])
    clusterings=[]

    #cluster
    while len(agenda)>minClusters:
      if len(agenda)<=maxClusters:
        clusterings.append([[],[],None])
        clusterings[-1][0]=[len(i[1]) for i in agenda]
        clusterings[-1][1]=[i[2] for i in agenda]
        clusterings[-1][2]=sum(clusterings[-1][1])/len(clusterings[-1][0])

      cluster=d.pop()
      clust=cluster[0]
      trees=cluster[1]
      cluster_diameter=0

      for i in range(len(cluster[1])):
        for j in range(i):
          if dist[cluster[1][i]][cluster[1][j]]>cluster_diameter:
            cluster_diameter=dist[cluster[1][i]][cluster[1][j]]

      new_clust=[clust,trees,cluster_diameter]

      agenda=[i for i in agenda if (i[0]!=new_clust[0][0] and i[0]!=new_clust[0][1])]
      d=[i for i in d if (new_clust[0][0] not in i[0] and new_clust[0][1] not in i[0])]

      for i in agenda:
        s=0
        for x in i[1]:
          for y in new_clust[1]:
            s+=dist[x][y]
        d.append([(i[0],new_clust[0]),i[1]+new_clust[1],s/(len(new_clust[1])*len(i[1]))])

      agenda.append(new_clust)
      d.sort(reverse=True,key=lambda x: x[2])

    if len(agenda)<=maxClusters:
      clusterings.append([[],[],None])
      clusterings[-1][0]=[len(i[1]) for i in agenda]
      clusterings[-1][1]=[i[2] for i in agenda]
      clusterings[-1][2]=sum(clusterings[-1][1])/len(clusterings[-1][0])
    return clusterings


These are the test cases for your `cluster` function.

In [38]:
def test_cluster():
    clusters = [[[3, 6, 3, 6, 2], [1, 2, 1, 2, 1], 1.4],
                [[9, 3, 6, 2], [2, 1, 2, 1], 1.5],
                [[9, 9, 2], [2, 3, 1], 2],
                [[18, 2], [5, 1], 3],
                [[20], [7], 7]]
    for i, clust in enumerate(cluster(treeList[:20], 1, 5)):
        print("Expected: " + str(clusters[i]) + "\nOutput:   " + str(clust))

In [45]:
test_cluster()

Expected: [[3, 6, 3, 6, 2], [1, 2, 1, 2, 1], 1.4]
Output:   [[2, 3, 3, 6, 6], [1, 1, 1, 2, 2], 1.4]
Expected: [[9, 3, 6, 2], [2, 1, 2, 1], 1.5]
Output:   [[2, 3, 6, 9], [1, 1, 2, 2], 1.5]
Expected: [[9, 9, 2], [2, 3, 1], 2]
Output:   [[2, 9, 9], [1, 2, 3], 2.0]
Expected: [[18, 2], [5, 1], 3]
Output:   [[2, 18], [1, 5], 3.0]
Expected: [[20], [7], 7]
Output:   [[20], [7], 7.0]


C) You will implement a general consensus algorithm that will allow you to find a consensus tree for each cluster and measure the quality of the cluster by its "specificity" (how close it is to being a binary tree).

In [89]:
"""
Calculate the specificity of 'cluster' given a target 'threshold'

Arguments:
    cluster - list of trees
    threshold - float value of threshold
Returns:
    spec - float specificity of 'cluster' given 'threshold'
"""
def specificity(cluster, threshold):
    #label each leaf
    t=construct(cluster[0])
    agenda=t.traverse()
    lookup={}
    for i in agenda:
      lookup[i]=random.randint(1,100000)

    d={}
    for tree in cluster:
      t=construct(tree)
      _,d=partitions(t,lookup,d)

    consensus=0
    for partition in d:
      if d[partition]/len(cluster)>=threshold:
        consensus+=1
        
    spec=(consensus-2)/(len(agenda)-3)
    return spec

These are the test cases for your `specificity` function.

In [49]:
def test_specificity():
    print("Expected: 0.77083\tOutput: " + str(specificity(treeList, 1.0)))
    print("Expected: 0.91667\tOutput: " + str(specificity(treeList, 0.51)))

In [90]:
test_specificity()

Expected: 0.77083	Output: 0.7708333333333334
Expected: 0.91667	Output: 0.9166666666666666


# (6.878 only) Coalescent simulation

In this problem, we will simulate the coalescent process. Recall that this is the time-reversal of the Wright-Fisher process.

A) Write a program to simulate the coalescent process on a population of $2N$ alleles. Track the times of coalescent events starting from the initial generation until all alleles coalesce to a single ancestor. If we are tracking $k$ lineages, you should report $k - 1$ coalescent events.

Recall that the Wright-Fisher process assumes each allele in the next generation is sampled independently from all alleles in the current generation. We are now interested in the reverse, so we instead need to sample parents in the previous generation uniformly at random with replacement. Note that we are interested in the identities of the parents and not their ancestral alleles.

Run 1,000 trials with a population size of $N = 500$. Report the mean and standard deviation of the number of generations between coalescent events for $k = 2$, $3$, $4$ and $5$ lineages.

In [ ]:
### YOUR CODE HERE ###

**Answer here:**

|Lineages|Mean|Standard Deviation|
|-|-|-|
|2|||
|3|||
|4|||
|5|||

B) Recall the waiting time between coalescent events is approximately exponentially distributed with parameter $\lambda$. For each value of $k$, what is the value of $\lambda$ given $N = 500$?

Assuming this distribution, the mean waiting time and its standard deviation are both $\frac{1}{\lambda}$. How do these expected values compare to your observed values? If your observed values are different, give an explanation of what could have caused the differences.

**Answer here:**

|Lineages|$\lambda$|$\frac{1}{\lambda}$|
|-|-|-|
|2|||
|3|||
|4|||
|5|||

C) Extend your simulator to model sexual reproduction.

Assume a fixed number of females $F$ (and therefore $M = N - F$ males) in each generation and that each chromosome in the next generation is selected in the following way: sample a male and female to mate uniformly at random, then sample one of the two alleles uniformly at random. Your simulation should do
the reverse: sample a father and mother and then pick one at random as the ancestor for each allele.

Run 1,000 trials with $F = 100$ and $M = 400$. Do your results agree with the coalescent approximation? Justify your results.

Can you extend the coalescent approximation to more accurately reflect this model of sexual reproduction? Do your results agree with this new approximation?

In [ ]:
### YOUR CODE HERE ###

**Answer here:**

|Lineages|Mean|Standard Deviation|
|-|-|-|
|2|||
|3|||
|4|||
|5|||